In [1]:
import pandas as pd
# import streamlit as st
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
file = r'C:/KKK/3-Coding/Dataset/OnlineretBERSIHfra.xlsx'

In [ ]:
df = pd.read_excel(file)
df

In [4]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
df

In [ ]:
basket_1 = (df
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
basket_1.head()

In [ ]:
def hot_encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_encoded = basket_1.applymap(hot_encode) 
basket_1 = basket_encoded 
basket_1.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
basket_1.head()

In [ ]:
frq_items = apriori(basket_1, min_support = 0.1, use_colnames = True) 
frq_items[''] = range(1, len(frq_items) + 1)
frq_items = frq_items[[''] + [col for col in frq_items.columns if col != '']]
frq_items

In [ ]:
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.85) 
rules = rules.sort_values(['confidence'], ascending =[False])
rules = rules.drop(columns=['leverage', 'conviction'])
rules[''] = range(1, len(rules) + 1)
rules = rules[[''] + [col for col in rules.columns if col != '']]
rules